<h2>209 - Uswatun Khasanah<h2>

# **Kegiatan Modul 6**

Content Based Filtering : Anime

Tulislan code sesuai instruksi pada sel yang telah disediakan

## Downlaod dan extract dataset dari situs [kaggle.com](https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020 -p /content/drive/MyDrive/dataset

Dataset URL: https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020
License(s): CC0-1.0
anime-recommendation-database-2020.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -o -j /content/drive/MyDrive/dataset/anime-recommendation-database-2020.zip -d /content/drive/MyDrive/dataset/dataset_anime

Archive:  /content/drive/MyDrive/dataset/anime-recommendation-database-2020.zip
  inflating: /content/drive/MyDrive/dataset/dataset_anime/anime.csv  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/anime_with_synopsis.csv  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/animelist.csv  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/details.html  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/pictures.html  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/recomendations.html  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/reviews_1.html  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/reviews_10.html  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/reviews_11.html  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/reviews_12.html  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/reviews_13.html  
  inflating: /content/drive/MyDrive/dataset/dataset_anime/reviews_14.html  
  inflatin

## Import library dan load dataset

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
file_path = '/content/drive/MyDrive/dataset/dataset_anime/anime_with_synopsis.csv'
anime_data = pd.read_csv(file_path)

In [ ]:
display(anime_data.head())

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


## Tangani data yang kosong (missing value)

In [ ]:
anime_data.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    8
dtype: int64

In [ ]:
data_bersih = anime_data.dropna()
anime_data.dropna()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


Hasil penanganan missing value

In [ ]:
anime_data = data_bersih
anime_data.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    0
dtype: int64

## Selanjutnya lakukan feature extraction pada fitur "Genres" menggunakan TF-IDF

In [ ]:
anime_data['Genres'] = anime_data['Genres'].fillna('')

tfidf_vectorizer = TfidfVectorizer(max_features=45)

tfidf_matrix = tfidf_vectorizer.fit_transform(anime_data['Genres'])

In [ ]:
tfidf_matrix.shape

(16206, 45)

## Menghitung cosine similarity dari TF-IDF diatas

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim.shape

(16206, 16206)

## Membuat Content Based Recomender

In [ ]:
def recommend_anime(title, cosine_sim=cosine_sim):
    idx = anime_data[anime_data['Name'] == title].index[0]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[0:5]

    anime_indices = [i[0] for i in sim_scores]

    return anime_data.iloc[anime_indices][['Name', 'Genres']]

## Mencoba Content Based Recomender

Mempilkan 5 Judul Anime dengan yang memiliki nilai kemiripan tertinggi berdasarkan genre dari kata kunci judul anime yang dimasukkan

In [ ]:
recommendations = recommend_anime('Paprika')
recommendations

,Name,Genres
1691,Paprika,"Dementia, Fantasy, Horror, Mystery, Psychologi..."
824,Higurashi no Naku Koro ni,"Mystery, Dementia, Horror, Psychological, Supe..."
15171,Higurashi no Naku Koro ni Gou,"Mystery, Dementia, Horror, Psychological, Supe..."
16211,Higurashi no Naku Koro ni Sotsu,"Mystery, Dementia, Horror, Psychological, Supe..."
13265,Yakusoku no Neverland,"Sci-Fi, Mystery, Horror, Psychological, Thrill..."


In [ ]:
recommendations = recommend_anime('Monster')
recommendations

,Name,Genres
9,Monster,"Drama, Horror, Mystery, Police, Psychological,..."
6598,Higurashi no Naku Koro ni Kaku: Outbreak,"Mystery, Horror, Psychological, Thriller"
2495,Death Note: Rewrite,"Mystery, Police, Psychological, Supernatural, ..."
10449,B: The Beginning,"Action, Mystery, Police, Psychological, Supern..."
13404,B: The Beginning Succession,"Action, Mystery, Police, Psychological, Supern..."


In [ ]:
recommendations = recommend_anime('Naruto')
recommendations

,Name,Genres
10,Naruto,"Action, Adventure, Comedy, Super Power, Martia..."
1508,Naruto: Shippuuden,"Action, Adventure, Comedy, Super Power, Martia..."
11640,Boruto: Jump Festa 2016 Special,"Action, Adventure, Comedy, Super Power, Martia..."
206,Rekka no Honoo,"Action, Adventure, Martial Arts, Shounen, Supe..."
5518,Naruto: Honoo no Chuunin Shiken! Naruto vs. Ko...,"Action, Adventure, Martial Arts, Shounen, Supe..."
